In [1]:
from lsm_tree_original import LSMTree
from tqdm import tqdm
from statistics import stdev, mean


# import cProfile
# import pstats
# import snakeviz

import pandas as pd
import pyarrow.parquet as pq
import sys
import numpy as np
import time
import random
import pickle
import matplotlib.pyplot as plt

In [2]:
df_parquet = pq.read_table('Data/2019-01-01_performance_fixed_tiles.parquet').to_pandas()

In [3]:
SEGMENTS_DIRECTORY_CKF = 'segments/cuckoo/'
SEGMENT_BASENAME_CKF = 'LSMTreeCuckoo'
WAL_BASENAME_CKF = 'wal_file_cuckoo'

SEGMENTS_DIRECTORY_BF = 'segments/bloom/'
SEGMENT_BASENAME_BF = 'LSMTreeBloom'
WAL_BASENAME_BF = 'wal_file_bloom'

size = sys.getsizeof(df_parquet[['quadkey','avg_d_kbps']])/1000000
print('Whole DataFrame      : ',size , len(df_parquet))

Whole DataFrame      :  395.040068 4877036


In [4]:
df_parquet['hashed_vals'] = df_parquet['tile'].apply(lambda x: abs(hash(x)))
df_parquet.head()

,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,hashed_vals
0,0231113112003202,"POLYGON((-90.6591796875 38.4922941923613, -90....",66216,12490,13,28,4,6292758158835805233
1,1322111021111001,"POLYGON((110.352172851562 21.2893743558604, 11...",102598,37356,13,15,4,3430291227720601726
2,3112203030003110,"POLYGON((138.592529296875 -34.9219710361638, 1...",24686,18736,18,162,106,1896895394411534007
3,0320000130321312,"POLYGON((-87.637939453125 40.225024210605, -87...",17674,13989,78,364,4,1735441524002766295
4,0320001332313103,"POLYGON((-84.7430419921875 38.9209554204673, -...",441192,218955,22,14,1,7258374205672227954


In [5]:
cardinality = df_parquet['hashed_vals'].nunique()/len(df_parquet)
cardinality

1.0

In [6]:
def insert_testing(lsm, df):
    start_time = time.time()
    for num, row in enumerate(df.iterrows()):   
        lsm.db_set(str(row[1][-1]), str(row[1][1]))   
    end_time = time.time()
    insertion_time = end_time - start_time
    return insertion_time

def query_testing(lsm, df, sample_num):
    random_samples = df.sample(n=sample_num)
    start_time = time.time()
    for row in random_samples.iterrows():   
        lsm.db_get(str(row[1][-1]))   
    end_time = time.time()
    query_time = end_time - start_time
    return query_time

In [7]:
time_th = 0.05

In [8]:
!rm segments/cuckoo/*
!rm segments/bloom/*
!rm segments/lsm_original/*

In [9]:
lsm_original = LSMTree()
lsm_original.set_time_threshold(time_th)

In [10]:
insertion_lsm_time = insert_testing(lsm_original, df_parquet.iloc[:100000])

In [11]:
query_lsm_time = query_testing(lsm_original, df_parquet.iloc[:100000], 10000)

In [12]:
insertion_lsm_time, query_lsm_time

(7.827477931976318, 73.23587536811829)

In [ ]:
insert_range        = [200000,  200000, 400000, 400000,  600000,   600000,  800000,  800000, 1000000, 1000000]
query_del_test_list = [15000,    30000,  45000,  60000,   75000,    90000,  105000,  120000,  135000,  150000] 
query_del_ranges    = ['15k',    '30k',  '45k',  '60k',   '75k',    '90k',  '105k',  '120k',  '135k',  '150k'] 
batch_size          = [20000,    20000,  40000,  40000,   60000,    60000,   80000,   80000,  100000,  100000] 
lvl_list            = [[10,20],[20,40],[40,80],[50,100],[60,120],[70,140],[80,160],[90,180],[100,190],[110,200]]

fpp = 0.1
df = df_parquet
time_th = 0.05

len(insert_range), len(query_del_test_list), len(query_del_ranges), len(batch_size), len(lvl_list)

query_bef_results = []
deletion_results = []
query_aft_results = []